In [1]:
import pandas as pd
import nltk
import re, random, os
from nltk.corpus import stopwords
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from gensim import corpora
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel

In [2]:
df = pd.read_csv('Samsung_reviews.csv')

In [3]:
df.shape

(300, 5)

In [4]:
df.head(10)

,Unnamed: 0,Model,rating,title,review_text
0,0,Samsung s20,5.0 out of 5 stars,5.0 out of 5 stars\nS20- Better than Ultra!!\n,\nI'm coming from S8. I didn't have any issue...
1,1,Samsung s20,1.0 out of 5 stars,1.0 out of 5 stars\nNothing could be worst exp...,\nNothing could be worst experience than this....
2,2,Samsung s20,5.0 out of 5 stars,"5.0 out of 5 stars\nGorgeous & ergonomic form,...","\nWhen you take a S20 into your hand, it is ba..."
3,3,Samsung s20,4.0 out of 5 stars,4.0 out of 5 stars\nGood camera phone with ave...,\nReview after 1.5 months of use. Body and loo...
4,4,Samsung s20,3.0 out of 5 stars,3.0 out of 5 stars\nReview After 4 Days.\n,\nI bought My S20 Last week its been 4 day I a...
5,5,Samsung s20,5.0 out of 5 stars,5.0 out of 5 stars\nPowerpacked: everything is...,\n*Reviewing after using it for 5 days*I had d...
6,6,Samsung s20,1.0 out of 5 stars,1.0 out of 5 stars\nDamaged product\n,\nIts damaged even before using....kindly canc...
7,7,Samsung s20,5.0 out of 5 stars,5.0 out of 5 stars\nMasterpiece Galaxy S 20\n,\nThis is my first premium phone and I am in L...
8,8,Samsung s20,3.0 out of 5 stars,3.0 out of 5 stars\nIt is a huge downgrade whe...,\nSo earlier samsung used to produce phones wi...
9,9,Samsung s20,5.0 out of 5 stars,5.0 out of 5 stars\nRead this review before yo...,\nFirst of all Delivery was really fast by ama...


In [5]:
import re

def remove_ratings(title):
    return re.sub(r'\d+\.\d+ out of \d+ stars', '', title).strip()

In [6]:
df['title'] = df['title'].apply(remove_ratings)

In [7]:
df['reviews'] =df['review_text']

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mohdzahan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/mohdzahan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
emoji_pattern = re.compile(
    "["
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "\U00002702-\U000027B0"
    "\U000024C2-\U0001F251"
    "]+",
    flags=re.UNICODE
)

In [11]:

df['reviews'] = df['reviews'].astype(str)

In [12]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

def preprocess(text):
    text = text.replace('\n', '.')
    text = emoji_pattern.sub(r'', text)  
    text = re.sub(r'\d+', '', text)  
    text = re.sub(r'\s+', ' ', text)  
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()  
    text = ' '.join(word for word in text.split() if word not in stop_words)  
    return text

df['reviews'] = df['reviews'].apply(preprocess)

In [13]:
nlp = spacy.load('en_core_web_md', disable = ['parser', 'ner'])

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ']):
    output = []
    for text in texts:
        doc = nlp(text)
        output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return output

tokenized_reviews = lemmatization(df['reviews'].tolist())

In [ ]:
def remove_topic(tokenized_reviews):
    unwanted = ['phone', 'oneplus','mobile']
    for i in unwanted:
        while i in tokenized_reviews:
            tokenized_reviews.remove(i)
    return tokenized_reviews

tokenized_reviews = remove_topic(tokenized_reviews)
tokenized_reviews

In [14]:
print(tokenized_reviews)

[['phone', 'year', 'worth', 'screen', 'tall', 'phone', 'similar', 'size', 'phone', 'processor', 'smart', 'switch', 'phone', 'easy', 'samsung', 'flexible', 'lot', 'customization', 'screen', 'thing', 'beautiful', 'toned', 'useless', 'curved', 'side', 'screen', 'hole', 'front', 'great', 'refresh', 'rate', 'resolution', 'samsung', 'incredible', 'display', 'difference', 'resolution', 'refresh', 'rate', 'phone', 'camera', 'great', 'flagship', 'phone', 'good', 'tech', 'notice', 'minor', 'benefit', 'difference', 'thing', 'setting', 'great', 'picture', 'scenario', 'night', 'time', 'complaint', 'wireless', 'bloatware', 'quirky', 'thing', 'service', 'area', 'superior', 'many', 'carrier', 'able', 'gb', 'microsd', 'claim', 'well', 'g', 'general', 'stupid', 'reason', 'verizon', 'record', 'one', 'cost', 'saving', 'measure', 'phone', 'tolerable', 'tech', 'big', 'sibling', 'fraction', 'price', 'pocketable', 'form', 'factor', 'important', 'small', 'phone', 'top', 'line', 'flagship', 'screen', 'camera', 

In [15]:
final_reviews = [' '.join(review) for review in tokenized_reviews]

In [16]:
final_reviews

['phone year worth screen tall phone similar size phone processor smart switch phone easy samsung flexible lot customization screen thing beautiful toned useless curved side screen hole front great refresh rate resolution samsung incredible display difference resolution refresh rate phone camera great flagship phone good tech notice minor benefit difference thing setting great picture scenario night time complaint wireless bloatware quirky thing service area superior many carrier able gb microsd claim well g general stupid reason verizon record one cost saving measure phone tolerable tech big sibling fraction price pocketable form factor important small phone top line flagship screen camera processor phone mid tier price range thing gb factor extra take advantage regular user lack ram memory expansion little disappointing base model plenty general fantastic phone great price phone great hyper sensitive techie care concession day use',
 'thing camera quality wonderful refresh rate displ

In [17]:
'''criteria = ["battery", "storage", "cost", "performance", "camera", "quality", "value", "appearance"]

for review in tokenized_reviews:
    for criterion in criteria:
        if criterion not in review:
            review.append(criterion)'''

'criteria = ["battery", "storage", "cost", "performance", "camera", "quality", "value", "appearance"]\n\nfor review in tokenized_reviews:\n    for criterion in criteria:\n        if criterion not in review:\n            review.append(criterion)'

In [18]:
dictionary = corpora.Dictionary(tokenized_reviews)

In [19]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(final_reviews)

In [20]:
print(tfidf_matrix.shape)

(300, 1757)


In [21]:
corpus = gensim.matutils.Sparse2Corpus(tfidf_matrix, documents_columns=False)

In [22]:
id2word = dict((v, k) for k, v in tfidf_vectorizer.vocabulary_.items())

In [23]:
LDA = gensim.models.ldamodel.LdaModel

lda_model = LDA(corpus, num_topics=5, random_state=100, chunksize=1000, iterations=500,id2word=id2word,alpha = 'auto',eta='auto')

In [24]:
lda_model.print_topics()

[(0,
  '0.009*"phone" + 0.008*"good" + 0.006*"battery" + 0.005*"camera" + 0.005*"display" + 0.005*"iphone" + 0.004*"time" + 0.004*"user" + 0.004*"heavy" + 0.004*"day"'),
 (1,
  '0.009*"phone" + 0.008*"battery" + 0.005*"fingerprint" + 0.005*"good" + 0.005*"reader" + 0.005*"camera" + 0.004*"use" + 0.004*"day" + 0.004*"screen" + 0.004*"life"'),
 (2,
  '0.014*"good" + 0.011*"phone" + 0.009*"battery" + 0.006*"great" + 0.006*"camera" + 0.005*"iphone" + 0.004*"life" + 0.004*"app" + 0.004*"time" + 0.004*"day"'),
 (3,
  '0.011*"phone" + 0.010*"good" + 0.010*"camera" + 0.007*"battery" + 0.006*"quality" + 0.004*"great" + 0.004*"awesome" + 0.004*"size" + 0.004*"device" + 0.003*"time"'),
 (4,
  '0.012*"phone" + 0.008*"good" + 0.007*"battery" + 0.005*"camera" + 0.004*"performance" + 0.004*"great" + 0.004*"nice" + 0.004*"life" + 0.004*"day" + 0.003*"screen"')]

In [25]:

topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.009*"phone" + 0.008*"good" + 0.006*"battery" + 0.005*"camera" + 0.005*"display" + 0.005*"iphone" + 0.004*"time" + 0.004*"user" + 0.004*"heavy" + 0.004*"day"')
(1, '0.009*"phone" + 0.008*"battery" + 0.005*"fingerprint" + 0.005*"good" + 0.005*"reader" + 0.005*"camera" + 0.004*"use" + 0.004*"day" + 0.004*"screen" + 0.004*"life"')
(2, '0.014*"good" + 0.011*"phone" + 0.009*"battery" + 0.006*"great" + 0.006*"camera" + 0.005*"iphone" + 0.004*"life" + 0.004*"app" + 0.004*"time" + 0.004*"day"')
(3, '0.011*"phone" + 0.010*"good" + 0.010*"camera" + 0.007*"battery" + 0.006*"quality" + 0.004*"great" + 0.004*"awesome" + 0.004*"size" + 0.004*"device" + 0.003*"time"')
(4, '0.012*"phone" + 0.008*"good" + 0.007*"battery" + 0.005*"camera" + 0.004*"performance" + 0.004*"great" + 0.004*"nice" + 0.004*"life" + 0.004*"day" + 0.003*"screen"')


In [ ]:
topic_list = []

for topic in topics:
    topic_num, topic_words = topic
    words = [word.split('*')[1].strip('"') for word in topic_words.split(" + ")]
    topic_list.append(words)

topic_lda_Samsung=[]
[topic_lda_Samsung.extend(k) for k in topic_list ]
topic_lda_Samsung

In [ ]:
def remove_topic(topic_lda_Samsung):
    t = ['phone', 'oneplus','mobile']
    for i in t:
        while i in topic_lda_Samsung:
            topic_lda_Samsung.remove(i)
    return topic_lda_Samsung

topic_lda_Samsung = remove_topic(topic_lda_Samsung)
topic_lda_Samsung

In [26]:
coherence_model_lda = CoherenceModel(
    model=lda_model, 
    texts=tokenized_reviews, 
    dictionary=dictionary, 
    coherence='c_v'
)
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.7688737570522342
